In [ ]:
import numpy as np 
import pandas as pd
import cv2
import os
import time
import shutil
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm
import tensorflow as tf
import tensorflow_hub as hub
import keras 
from keras.applications import VGG16
from keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
from keras.applications.densenet import preprocess_input as preprocess_input_densenet
from keras.layers.experimental.preprocessing import Rescaling
from keras.layers import Input, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
import pickle
from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing.image import image_dataset_from_directory

import seaborn as sns
from scipy import signal
from scipy import ndimage
# 
import copy
import warnings
from sklearn.utils import class_weight

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
categories = ['Baeolophusbicolorq_A', 'Cardellinacanadensisq_A', 'Cardinaliscardinalisq_A', 'Catharusminimusq_A', 'Catharusustulatusq_A', 'Geothlypisphiladelphiaq_A', 'Geothlypistrichasq_A',
              'Melanerpeserythrocephalusq_A', 'Passerculussandwichensisq_A', 'Pheucticusludovicianusq_A', 'Poecileatricapillusq_A', 'Seiurusaurocapillaq_A', 'Setophagacaerulescensq_A',
              'Setophagacastaneaq_A', 'Setophagaruticillaq_A', 'Spinustristisq_A', 'Spizellapasserinaq_A', 'Spizelloidesarboreaq_A', 'Zonotrichiaalbicollisq_A']




my_files = r'../input/new-england-labeled-bird-species-spectrograms/LabeledSpectrograms'



image_generator = ImageDataGenerator(rescale=1/256)

full_dataset = image_generator.flow_from_directory(batch_size=32,
                                                    directory=my_files,
                                                    shuffle=True,
                                                    target_size=(251, 251),
                                                    classes=categories,
                                                   color_mode='grayscale')


classes = np.unique(full_dataset.classes)
_class_weight = class_weight.compute_class_weight('balanced',classes,full_dataset.classes)

class_weights = {}
i = 0
for i in range(len(classes)):
    class_weights[classes[i]] = _class_weight[i]
    i+=1



image_generator = ImageDataGenerator(validation_split=0.1, rescale=1/256)
image_generator2 = ImageDataGenerator(validation_split=0.1, rescale=1/256, horizontal_flip=False)

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                    directory=my_files,
                                                    shuffle=True,
                                                    target_size=(251, 251),
                                                    subset='training',
                                                    classes=categories)

test_dataset = image_generator2.flow_from_directory(batch_size=32,
                                                    directory=my_files,
                                                    shuffle=True,
                                                    target_size=(251, 251),
                                                    subset='validation',
                                                    classes=categories)

In [ ]:
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

# Define my model
URL = r'https://tfhub.dev/google/imagenet/resnet_v2_152/feature_vector/4'
# URL = r'https://tfhub.dev/agripredict/disease-classification/1'
# URL = r'https://tfhub.dev/google/bit/m-r152x4/imagenet21k_classification/1'


feature_extractor = hub.KerasLayer(URL, input_shape=(224,224,3))
feature_extractor.trainable = False

deep_model = tf.keras.models.Sequential([    
    feature_extractor,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(19,activation='sigmoid')    
])
deep_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='CategoricalCrossentropy', metrics=METRICS )
deep_model.summary()

# deep_model = tf.keras.models.Sequential([
    
#     tf.keras.Input(shape=(251, 251, 1)),
#     tf.keras.layers.Conv2D(64, 5, strides=2, activation="relu"),
#     tf.keras.layers.Conv2D(64, 3, strides=2, activation="relu"),
#     tf.keras.layers.MaxPooling2D(3),
#     tf.keras.layers.Conv2D(64, 3, activation="relu"),
#     tf.keras.layers.Conv2D(64, 3, activation="relu"),
#     tf.keras.layers.MaxPooling2D(3),
#     tf.keras.layers.Conv2D(64, 3, activation="relu"),
#     tf.keras.layers.MaxPooling2D(2),
#     tf.keras.layers.GlobalMaxPooling2D(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(19,activation='sigmoid')  
    
              
# #     tf.keras.layers.Conv2D(38, 8, strides=1, activation="relu"),
# #     tf.keras.layers.MaxPooling2D(2),
# #     tf.keras.layers.GlobalMaxPooling2D(),
# #     tf.keras.layers.Dense(19,activation='sigmoid')   
# ])

# deep_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.000000001), loss='CategoricalCrossentropy', metrics=METRICS)
# # deep_model.build((None, 251, 251, 1))
# deep_model.summary()


In [ ]:
# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self,epoch,logs={}):
#     if(logs['accuracy']>=0.99):
#       self.model.stop_training=True

# callbacks=myCallback()

# checkpoint_callback = ModelCheckpoint('model1', monitor='val_accuracy', verbose = True, save_best_only = True, save_weights_only = False, mode= 'max')
# early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=3)

# METRICS = [
#         'accuracy',
#         tf.keras.metrics.Precision(name='precision'),
#         tf.keras.metrics.Recall(name='recall')
#     ]
# model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='CategoricalCrossentropy', metrics=METRICS )

# # How many epochs should I use?
# history = model.fit(train_dataset, epochs=20 , callbacks=[callbacks, checkpoint_callback, early_stopping_callback], validation_data=test_dataset)


deep_checkpoint_callback = ModelCheckpoint('deep_model_augs.h5', monitor='val_accuracy', verbose = True, save_best_only = True, save_weights_only = False, mode= 'max')
deep_early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10)
with tf.device('/GPU:0'):
    deep_fit = deep_model.fit(train_dataset,
                              class_weight=class_weights,
                              validation_data=test_dataset,
                              steps_per_epoch= (train_dataset.samples // 32),
                              validation_steps = (test_dataset.samples // 32),
                              use_multiprocessing=True,
                              workers=-1,
                              verbose = True,
                              epochs = 20,
                              callbacks = [deep_checkpoint_callback, deep_early_stopping_callback])

In [ ]:
plt.plot(np.array(deep_fit.history['loss']),color='blue')
plt.plot(np.array(deep_fit.history['val_loss']),color='red')
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(np.array(deep_fit.history['accuracy']),color='blue')
plt.plot(np.array(deep_fit.history['val_accuracy']),color='red')
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
predictions = deep_model.predict(test_dataset)
predictions = tf.argmax(predictions, axis=-1)
# predictions
actuals = test_dataset.classes

cm = tf.math.confusion_matrix(actuals, predictions)
cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

plt.figure(figsize=(20, 20))
sns.heatmap(
    cm, annot=True,
    xticklabels=categories,
    yticklabels=categories)
plt.xlabel("Predicted")
plt.ylabel("True")